![king](https://kingcounty.gov/img/KClogo-horiz-black-lg.gif) 
# EDA Project - [Housing Market in King County (WA, USA)](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction)

---
### Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import osmnx as ox
import geojson
from keplergl import KeplerGl
import warnings
warnings.filterwarnings('ignore')

---
### Data Exploration

Create a Pandas Dataframe and get a feel of the data

In [ ]:
df = pd.read_csv('./data/King_County_House_prices_dataset.csv')

In [ ]:
df.info()

#### Column Names and descriptions for King County Data Set
- **id** - unique id for each house
- **date** - date of sale
- **price** - is prediction target
- **bedrooms** - # of bedrooms
- **bathrooms** - # of bathrooms
- **sqft_living** - footage of the home
- **sqft_lot** - footage of the lot
- **floors** - floors (levels) in house
- **waterfront** - House which has a view to a waterfront
- **view** - An index from 0 to 4 of how good the view of the property was
- **condition** - How good the condition is ( Overall )
- **grade** - overall grade given to the housing unit, based on King County grading system
- **sqft_above** - square footage of house apart from basement
- **sqft_basement** - square footage of the basement
- **yr_built** - year house was built
- **yr_renovated** - year when house was renovated
- **zipcode** - zipcode area
- **lat** - Latitude coordinate
- **long** - Longitude coordinate
- **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
- **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors

In [ ]:
# Count the number of NaN in each column
nans = pd.DataFrame(df.isnull().sum(),columns=['count'])
nans.query('count != 0')

In [ ]:
df.describe().round(1)

33 bedrooms seems a lot. Let's investigate further:

In [ ]:
df.query('bedrooms == 33.0')

That many bedrooms in a 1620 ft² home? This has to be a typo. Let's drop that entry.

In [ ]:
df.drop(15856, axis=0, inplace=True)

Let's get a quick correlation overview:

In [ ]:
sns.pairplot(df)

Pretty convoluted, so let's try something different:

In [ ]:
plt.figure(figsize=(20, 15))
cor_mask = np.triu(df.corr())
ax = sns.heatmap(round(df.corr()*100), annot=True, mask=cor_mask, cmap='viridis')

Much better but nothing too exciting here. Everything is as you would expect.
* 'sqft_living' correlates highly with 'sqft_above' because people don't live in underground bunkers. 
* 'yr_built' correlates negatively with 'condition'. The older the house, the worse the condition it is in.
* Price ('price) increases with the size of the house ('sqft_living') which also means more bathrooms and bedrooms, but not much with the size of the property (sqft_lot).

Interestingly it seems that newer houses have more floors. It could be that multi-story buildings are built to accommodate increasing population and limited building space.

### Research Questions

| Question | Hypothesis | Indicators |
| -------- | ---------- | --------- |
| Q1 Where are the larger properties located?| Larger properties are located outside the city. | zipcode, sqft_lot |
| Q2 Are 'nicer' houses more expensive | Yes. |  'condition','grade', 'price' |
| Q3 Are bigger houses more expensive? | Yes. | 'price', 'sqft_living' |

__Q1) Where are the larger properties located?__

For the following step you need to download the [zipcodes geojson file](https://data.amerigeoss.org/dataset/zip-codes-2360f/resource/3aeb8b59-92d9-43a6-b959-28b7ee324866) and place it in the data folder.

In [ ]:
# THANKS MALTE!
#load geojson with zipcodes
with open('./data/zip_codes.geojson') as f:
    zipson = geojson.load(f)

# adding zipcodes as 'id' for each zipcode area in the geojson, so plotly can link zip code data to appropriate area
for i in range(len(zipson['features'])):
    zipson['features'][i]['id'] = zipson['features'][i]['properties']['ZIPCODE']

# group by zipcode and calculate the mean
q1 = df.groupby('zipcode')
q1 = q1.mean().reset_index()

In [ ]:
# plot the average plot size by zip code
fig = px.choropleth_mapbox(q1, geojson=zipson, locations='zipcode', 
    color='sqft_lot',
    color_continuous_scale='Viridis',
    range_color=(3000, 100000),
    mapbox_style='carto-positron',
    zoom=8,
    center={'lat':47.460, 'lon':-122.014},
    opacity=0.5,
    hover_data=['sqft_lot', 'zipcode'],
    labels={'sqft_lot':'avg lot size[ft²]'}
)
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0,})
fig.show()

The map above shows that the further you are from the Seattle city center the average lot size increases. This could be due to agricultural use of the land and the sales date from this area includes some farms. Some zipcode areas are missing. Possibly because no houses were sold in those areas in the time frame the data was collected. Interesting is the big average lot size on the island of Vashon west of Seattle.

__Q2) Are 'nicer' houses more expensive?__

In [ ]:
# group data by condition and calculate the average
q2=df.groupby(['condition']).mean()['price'].reset_index()

plt.figure(figsize=(20,10))
sns.barplot(x='condition', y='price', data=q2)
plt.ylabel('Avg. price in $')
plt.xlabel('Condition (higher is better)')
plt.title('Average price in relation to condition')
plt.show();


There is a clear division in the average sale prices of houses with a condition of 1-2 and 3-5. The differences within those two groups are less pronounced.

In [ ]:
sns.set_theme(style="darkgrid")
plt.figure(figsize=(20,10))
sns.boxplot(x=df.grade, y=df.price)
plt.ylabel('Price in M$')
plt.xlabel('Grade')
plt.title('Price in relation to grade')
plt.show();

The boxplot above shows that the price increases with the grade of the house. The number of outliers in the grade range from 7 to 10 could mean, that a lot of sellers sold their house above market price. This may indicate a boom in the region, caused by big tech companies like Amazon and Google.

__Q3) Are bigger houses more expensive?__

In [ ]:
plt.figure(figsize=(20,10))
sns.regplot(data=df, x='sqft_living', y='price', line_kws={'color': 'red'})
plt.ylabel('Price in M$')
plt.xlabel('Living space in ft²')
plt.title('Price in relation living space')
plt.show();

Not really an interesting question.
It's clear as day that houses with larger living space are more expensive

---
### Client

<img src="./images/scrooge.png" alt="drawing" width="200"/>

Jabob Philips' mansion requirements:
* Unlimited Budget
* 4+ bathrooms or smaller house nearby
* big lot (tennis court & pool), golf
* historic
* no waterfront view

### Workflow:

1. Drop dataframe columns that are not needed for our analysis.

2. Filter by 'hard' requirements:
* filter by waterfront
* filter by bathrooms or small house nearby

3. Filter by 'soft' requirements
* filter by lot size so the it at least fits a tennis court and a pool
* make sure that the prospective house are up to the client's standard
    
4. Get a list of golf courses that are in the area

5. Get a list of historical buildings

6. Plot a map the get the final candidates to present to the client

### 1) Drop dataframe columns that are not needed for our analysis.

Get a smaller dataframe to work with, with only the columns that are of interest:

| Requirement | Columns of interest |
|---|---|
|no waterfront| 'waterfront'|
|4+ bathrooms|'bathrooms', 'sqft_living15'|
|big lot|'sqft_lot' |
|historic| 'lat', 'lon', ('yr_built')|
|Unlimited Budget|'price' (not really, but..)|
| Luxurious | 'grade', 'condition' |

as well as general columns like 'id'


In [ ]:
# Create dataframe with columns of interest
small_df = df[['id','price','waterfront','bathrooms', 'bedrooms', 'sqft_living', 'sqft_lot','sqft_living15','yr_built','lat','long', 'grade', 'condition']]

We saw before that the 'waterfront' column has some NaN. Let's investigate further:

In [ ]:
small_df['waterfront'].unique()

Alright. Seems like a binary column. Makes sense. Either the property has a waterfront view or not. But some entries are missing :(

Let's table that for later.

### 2) Filter by 'hard' requirements:

In [ ]:
# only properties that do not have a waterfront view or NaN
f_water_df = small_df.query('waterfront == 0.0 or waterfront.isnull()')
f_water_df

What constitutes a smaller house? 

In [ ]:
# smaller than the mean?
print(df['sqft_living15'].mean().round(1))
# lower 20%?
print(df['sqft_living15'].quantile(q=0.2))

Since we wouldn't categorize houses that are technically smaller than the mean but still close to it as small, we use the lower 20%.

In [ ]:
f_bath = f_water_df.query('bathrooms >= 4.00 or sqft_living15 < 1420').sort_values('sqft_living')
f_bath

### 3) Filter by 'soft' requirements

Filter by lot size:
Our client requires space for at least a double tennis court (__2,808 ft²__) and a swimming pool (__~2000 ft²__).
Plus a decently sized garden this amounts to roughly __25,000f t²__. Another way of making sure our client's needs are fulfilled is by checking the top 5% of properties on the market.

In [ ]:
# top 5% of houses by lot size
df['sqft_lot'].quantile(q=0.95)

This should be enough be enough space to multiple tennis courts and a olympic size pool plus a hedge maze.


In [ ]:
f_sqft = f_bath.query('sqft_lot > 43308.75').sort_values('price', ascending=False)
f_sqft

Since the client is a man of refined taste, we don't want to present him with a sub par house. Let's get rid of all the low quality houses that are left by using a new category called 'luxury'. It is the product of 'condition' and 'grade'. The luxury score should at least be a __40__ (condition 4 * grade 10). Explanation [here](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?.type=r#g).

([Backup link for grade](./images/grade.png) and [backup link for condition](./images/condition.png))

In [ ]:
f_sqft.eval('luxury = grade*condition ', inplace=True)
f_luxury = f_sqft.query('luxury >= 40').sort_values('luxury', ascending=False)
f_luxury


And here we have our shortlist of candidates! And we have no entries in the waterfront column that are 'NaN'. Lucky :four_leaf_clover: !

### 4) Get a list of golf courses that are in the area

_Data Source: OpenStreetMaps_ and [Tagfinder](https://tagfinder.herokuapp.com/)

In [ ]:
# find features using tagfinder. golf courses are under the category 'leisure'
tags = {'leisure': 'golf_course'}
# download all golf courses in a area around Seattle and save them in a geopandas dataframe
gdf_golf = ox.geometries.geometries_from_bbox(48.0761, 47.011, -120.788, -123.0344, tags)
gdf_golf_slim = gdf_golf[['name', 'geometry']].reset_index() #throw away everything except name and location
gdf_golf_poly =  gdf_golf_slim[gdf_golf_slim.geometry.type=="Polygon"]

### 5) Get a list of historical buildings

_Data Source: National Register of Historic Places [in Seattle](https://en.wikipedia.org/wiki/National_Register_of_Historic_Places_listings_in_Seattle) and [around Seattle](https://en.wikipedia.org/wiki/National_Register_of_Historic_Places_listings_in_King_County,_Washington)
retrieved from Wikipeda_

In [ ]:
# download .kml files using the links above, read both files into geopandas dataframes and concat
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gdf1 = gpd.read_file('./data/king_county1.kml', driver='KML')
gdf2 = gpd.read_file('./data/king_county2.kml', driver='KML')
historic_gdf = pd.concat([gdf1, gdf2])

It might be useful to convert our list of candidates into geopandas dateframe for plotting later:

In [ ]:
# make geodataframe
gdf_luxury = gpd.GeoDataFrame(f_luxury, geometry=gpd.points_from_xy(x=f_luxury.long, y=f_luxury.lat))
gdf_luxury

### 6) Plot a map the get the final candidates to present to the client

In [ ]:
# convert geodataframe to geojson and write to file. already done. files are in the data folder.
#gdf_golf_slim.to_file('./data/golf_courses.json', driver="GeoJSON")
#historic_gdf.to_file('./data/historic_houses.json', driver="GeoJSON")
#gdf_luxury.to_file('./data/valid_properties.json', driver="GeoJSON")

In [ ]:
# Initialize map
map = KeplerGl(height=800)

#open .json files and add to map
with open('./data/golf_courses.json', 'r') as f:
    golf_courses = f.read()
with open('./data/historic_houses.json', 'r') as f:
    historic_houses = f.read()
with open('./data/valid_properties.json', 'r') as f:
    valid_properties = f.read()

map.add_data(data=golf_courses, name='golf_courses')
map.add_data(data=historic_houses, name='historic_houses')
map.add_data(data=valid_properties, name='valid_properties')

#display map
#%run ./data/hex_config.py
map

In [ ]:
# save config to file
# with open('hex_config.py', 'w') as f:
#    f.write('config = {}'.format(map.config))

Unfortunately, none of the properties for sale are classified as historic.

Since Seattle has a surprising amount of golf courses. All of the properties are close to one of them!

Upon inspecting the list of valid properties with satellite imagery we are able to recommend to following three houses to our client:

In [ ]:
gdf_luxury.query('id == 3425069083 or id == 3304700130 or id == 4054500390').sort_values('sqft_lot', ascending=False)

---

### Conclusion

Based on the requirements set by the client we are able to present him with three very good options for purchase. Although none of them are officially recognized as historic, they are all close to a golf course, have four or more bathrooms, no direct view of a natural body of water, and a large enough lot to build a pool and a tennis court if not already present.


First Choice!
* Pool and Tennis court already included! 
* Gigantic lot size with 284011 ft² and a living area of 4225 ft².

![firstchoice](./images/3425069083.png)

Second Choice!

* Tennis Court included!
* Very close to the Seattle Golf Club!
* Built in 1927, so almost 100 years old. Might be considered historic by some!


![secondchoice](./images/3304700130.png)

Third Choice!

* Largest living area of the three with massive 5310 ft².
* Enough space to build the pool and/or tennis court of your dreams!

![thirdchoice](./images/4054500390.png)